In [314]:
import pandas as pd
pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns", 6000)
# 유저 데이터 불러오기
user_data = pd.read_csv("./ml-100k/u.user", sep="|", names=["id", "age", "gender", "occupation", "zip_code"], index_col="id")
# movie 정보의 경우 컬럼수가 많으므로 따로 정의
movie_col_list = ["movie_id", "movie_title", "release_date", "video_release_date",
              "IMDb_URL", "unknown", "Action", "Adventure", "Animation",
              "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
              "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
              "Thriller", "War", "Western"]
# 영화 테이블 불러오기
movie_data = pd.read_csv("./ml-100k/u.item", sep="|", names=movie_col_list, index_col="movie_id")

# 평가 테이블 불러오기
rating_data = pd.read_csv("./ml-100k/u.data", sep="\t", names=["user_id","movie_id","rating", "timestamp"])

# 장르 정보 테이블 불러오기
genre_info = pd.read_csv("./ml-100k/u.genre", sep="|", names=["genre_name", "genre_id"], index_col="genre_id")


In [315]:
#영화의 장르를 길게 늘어뜨린 형식이 아닌 1:N으로 받기 위한 테이블을 새로 생성(movie_genre)


genre_names = genre_info["genre_name"].values


dict_movie_genre = {"movie_id": [], "genre_id":[]}
for i in range(len(genre_info)):
    for j in range(1, len(movie_data)+1):
        if movie_data[genre_info["genre_name"][i]][j]:
            dict_movie_genre["movie_id"].append(j)
            dict_movie_genre["genre_id"].append(i)
            
movie_genre = pd.DataFrame(data=dict_movie_genre).sort_values(by=["movie_id","genre_id"]).reset_index().drop(columns=["index"])


In [250]:
# 베이스 테이블 (유저, 영화, 평점 조인)
joined_user_rating = pd.merge(left=user_data, right=rating_data, left_on="id", right_on="user_id")
joined_user_rating_movie = pd.merge(left=joined_user_rating, right=movie_data, on="movie_id")
joined_user_rating_movie


,age,gender,occupation,zip_code,user_id,movie_id,rating,timestamp,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,24,M,technician,85711,1,61,4,878542420,Three Colors: White (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Trzy%20kolory...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,47,M,educator,29206,13,61,4,882140552,Three Colors: White (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Trzy%20kolory...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,35,F,other,37212,18,61,4,880130803,Three Colors: White (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Trzy%20kolory...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,27,M,programmer,52246,58,61,5,884305271,Three Colors: White (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Trzy%20kolory...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,49,M,educator,08403,59,61,4,888204597,Three Colors: White (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Trzy%20kolory...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,17,M,student,60089,863,1679,3,889289491,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
99996,17,M,student,60089,863,1678,1,889289570,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
99997,17,M,student,60089,863,1680,2,889289570,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
99998,28,M,writer,91505,896,1681,3,887160722,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [239]:
# 과제 2-A-1: 영화 평가 개수 top100 - 모든 유저
top_100_count_by_all_user = joined_user_rating_movie.groupby("movie_title")["rating"].count()
top_100_count_by_all_user = pd.DataFrame(top_100_count_by_all_user).rename(columns={"rating":"rating_count"})
top_100_count_by_all_user = top_100_count_by_all_user.sort_values(by="rating_count", ascending=False).head(100)
top_100_count_by_all_user

,rating_count
movie_title,
Star Wars (1977),583
Contact (1997),509
Fargo (1996),508
Return of the Jedi (1983),507
Liar Liar (1997),485
"English Patient, The (1996)",481
Scream (1996),478
Toy Story (1995),452
Air Force One (1997),431


In [171]:
def top(df, column: str, n: int, asc=True):
    return df.sort_values(by=column, ascending=asc)[:n]


In [243]:
# 과제 2-A-2: 영화 평가 개수 각 top100 - 성별별로
top_100_count_by_gender = joined_user_rating_movie.groupby(["gender", "movie_title"])[["rating"]].count()
top_100_count_by_gender = top_100_count_by_gender.rename(columns={"rating": "rating_count"})
top_100_count_by_gender = top_100_count_by_gender.sort_values(by=["gender","rating_count"], ascending=False)
# 각 그룹별로 100개씩 가져오기
top_100_count_by_gender = top_100_count_by_gender.groupby("gender").apply(top, column="rating_count", asc=False, n=100)
top_100_count_by_gender

rating_count
gender gender movie_title                                                     
F      F      English Patient, The (1996)                                  152
              Star Wars (1977)                                             151
              Scream (1996)                                                143
              Liar Liar (1997)                                             141
              Contact (1997)                                               137
              Air Force One (1997)                                         126
              Fargo (1996)                                                 125
              Return of the Jedi (1983)                                    124
              Toy Story (1995)                                             119
              Jerry Maguire (1996)                                         116
              Independence Day (ID4) (1996)                                106
              Godfather, The (1972)                                        105
              Titanic (1997)                                               104
              Silence of the Lambs, The (1991)                             100
              Full Monty, The (1997)                                        98
              Truth About Cats & Dogs, The (1996)                           97
              Raiders of the Lost Ark (1981)                                95
              Sense and Sensibility (1995)                                  95
              Chasing Amy (1997)                                            94
              Willy Wonka and the Chocolate Factory (1971)                  93
              Mr. Holland's Opus (1995)                                     92
              Empire Strikes Back, The (1980)                               91
              Rock, The (1996)                                              91
              Back to the Future (1985)                                     90
              Princess Bride, The (1987)                                    89
              Birdcage, The (1996)                                          88
              When Harry Met Sally... (1989)                                87
              E.T. the Extra-Terrestrial (1982)                             87
              Conspiracy Theory (1997)                                      85
              Saint, The (1997)                                             85
              Pulp Fiction (1994)                                           84
              Dead Man Walking (1995)                                       83
              Evita (1996)                                                  83
              Twister (1996)                                                83
              Twelve Monkeys (1995)                                         82
              Men in Black (1997)                                           80
              Schindler's List (1993)                                       79
              Forrest Gump (1994)                                           79
              Indiana Jones and the Last Crusade (1989)                     78
              Mission: Impossible (1996)                                    78
              Amadeus (1984)                                                77
              L.A. Confidential (1997)                                      76
              In & Out (1997)                                               75
              Leaving Las Vegas (1995)                                      75
              Sound of Music, The (1965)                                    75
              Dante's Peak (1997)                                           73
              Star Trek: First Contact (1996)                               73
              Phenomenon (1996)                                             72
              Ransom (1996)                                                 72
              Fugitive, The (1993)   

In [244]:
# 과제 2-A-3: 영화 평가 개수 각 top100 - 직업별로
top_100_count_by_occupation = joined_user_rating_movie.groupby(["occupation", "movie_title"])[["rating"]].count()
top_100_count_by_occupation = top_100_count_by_occupation.rename(columns={"rating": "rating_count"})
top_100_count_by_occupation = top_100_count_by_occupation.sort_values(by=["occupation","rating_count"], ascending=False)
# 각 그룹별로 100개씩 가져오기
top_100_count_by_occupation = top_100_count_by_occupation.groupby("occupation").apply(top, column="rating_count", asc=False, n=100)
top_100_count_by_occupation

rating_count
occupation    occupation    movie_title                                                     
administrator administrator English Patient, The (1996)                                   47
                            Star Wars (1977)                                              44
                            Return of the Jedi (1983)                                     40
                            Air Force One (1997)                                          38
                            Contact (1997)                                                38
                            Jerry Maguire (1996)                                          38
                            Liar Liar (1997)                                              37
                            Fargo (1996)                                                  36
                            Silence of the Lambs, The (1991)                              34
                            Scream (1996)                                                 33
                            Independence Day (ID4) (1996)                                 33
                            Full Monty, The (1997)                                        32
                            Godfather, The (1972)                                         32
                            Back to the Future (1985)                                     31
                            Pulp Fiction (1994)                                           31
                            Toy Story (1995)                                              31
                            Raiders of the Lost Ark (1981)                                29
                            Twelve Monkeys (1995)                                         29
                            Empire Strikes Back, The (1980)                               28
                            Alien (1979)                                                  27
                            In & Out (1997)                                               27
                            Mission: Impossible (1996)                                    27
                            Star Trek: First Contact (1996)                               27
                            Dead Man Walking (1995)                                       27
                            Chasing Amy (1997)                                            26
                            To Kill a Mockingbird (1962)                                  26
                            Twister (1996)                                                26
                            Fugitive, The (1993)                                          25
                            Sting, The (1973)                                             25
                            Indiana Jones and the Last Crusade (1989)                     25
                            2001: A Space Odyssey (1968)                                  25
                            Blade Runner (1982)                                           25
                            Birdcage, The (1996)                                          25
                            Mr. Holland's Opus (1995)                                     24
                            When Harry Met Sally... (1989)                                24
                            Rock, The (1996)                                              24
                            Men in Black (1997)                                           24
                            E.T. the Extra-Terrestrial (1982)                             24
                            Fish Called Wanda, A (1988)                                   24
                            Forrest Gump (1994)                                           24
                            Broken Arrow (1996)                                           23
                            Casablanca (1942)                                    

In [245]:
# 과제 2-B-1: 영화 평점 순 top 100 - 모든 유저
top_100_mean_by_all_user = joined_user_rating_movie.groupby("movie_title")["rating"].mean()
top_100_mean_by_all_user = pd.DataFrame(top_100_mean_by_all_user).rename(columns={"rating":"rating_mean"})
top_100_mean_by_all_user = top_100_mean_by_all_user.sort_values(by="rating_mean", ascending=False).head(100)
top_100_mean_by_all_user


,rating_mean
movie_title,
They Made Me a Criminal (1939),5.000000
Marlene Dietrich: Shadow and Light (1996),5.000000
"Saint of Fort Washington, The (1993)",5.000000
Someone Else's America (1995),5.000000
Star Kid (1997),5.000000
"Great Day in Harlem, A (1994)",5.000000
Aiqing wansui (1994),5.000000
Santa with Muscles (1996),5.000000
Prefontaine (1997),5.000000


In [247]:
# 과제 2-B-2: 영화 평점순 각 top100 - 각 성별 그룹 별
top_100_mean_by_gender = joined_user_rating_movie.groupby(["gender", "movie_title"])[["rating"]].mean()
top_100_mean_by_gender = top_100_mean_by_gender.rename(columns={"rating": "rating_mean"})
top_100_mean_by_gender = top_100_mean_by_gender.sort_values(by=["gender","rating_mean"], ascending=False)
# 각 그룹별로 100개씩 가져오기
top_100_mean_by_gender = top_100_mean_by_gender.groupby("gender").apply(top, column="rating_mean", asc=False, n=100)
top_100_mean_by_gender

rating_mean
gender gender movie_title                                                    
F      F      Everest (1998)                                         5.000000
              Someone Else's America (1995)                          5.000000
              Year of the Horse (1997)                               5.000000
              Visitors, The (Visiteurs, Les) (1993)                  5.000000
              Stripes (1981)                                         5.000000
              Telling Lies in America (1997)                         5.000000
              Prefontaine (1997)                                     5.000000
              Mina Tannenbaum (1994)                                 5.000000
              Maya Lin: A Strong Clear Vision (1994)                 5.000000
              Foreign Correspondent (1940)                           5.000000
              Faster Pussycat! Kill! Kill! (1965)                    5.000000
              Schindler's List (1993)                                4.632911
              Close Shave, A (1995)                                  4.631579
              Shawshank Redemption, The (1994)                       4.562500
              Wallace & Gromit: The Best of Aardman Animation...     4.533333
              Shall We Dance? (1996)                                 4.529412
              Some Mother's Son (1996)                               4.500000
              Women, The (1939)                                      4.500000
              Friday (1995)                                          4.500000
              Band Wagon, The (1953)                                 4.500000
              Wrong Trousers, The (1993)                             4.444444
              Some Folks Call It a Sling Blade (1993)                4.416667
              Sling Blade (1996)                                     4.411765
              Secrets & Lies (1996)                                  4.407407
              Casablanca (1942)                                      4.400000
              Christmas Carol, A (1938)                              4.400000
              Much Ado About Nothing (1993)                          4.381818
              Good Will Hunting (1997)                               4.355932
              Rear Window (1954)                                     4.351852
              To Kill a Mockingbird (1962)                           4.347826
              Anne Frank Remembered (1995)                           4.333333
              Hoop Dreams (1994)                                     4.333333
              Horseman on the Roof, The (Hussard sur le toit,...     4.333333
              Primary Colors (1998)                                  4.333333
              Safe (1995)                                            4.333333
              SubUrbia (1997)                                        4.333333
              Usual Suspects, The (1995)                             4.333333
              Silence of the Lambs, The (1991)                       4.320000
              Persuasion (1995)                                      4.315789
              Thin Man, The (1934)                                   4.307692
              Manchurian Candidate, The (1962)                       4.304348
              Dr. Strangelove or: How I Learned to Stop Worry...     4.291667
              It's a Wonderful Life (1946)                           4.288136
              Affair to Remember, An (1957)                          4.285714
              Antonia's Line (1995)                                  4.285714
              Higher Learning (1995)                                 4.285714
              Paradise Lost: The Child Murders at Robin Hood ...     4.285714
              Philadelphia Story, The (1940)                         4.281250
              Titanic (1997)                                         4.278846
              12 Angry Men (1957)                                    4.269231
          

In [246]:
# 과제 2-B-3: 영화 평점순 각 top100 - 각 직업 그룹 별
top_100_mean_by_occupation = joined_user_rating_movie.groupby(["occupation", "movie_title"])[["rating"]].mean()
top_100_mean_by_occupation = top_100_mean_by_occupation.rename(columns={"rating": "rating_mean"})
top_100_mean_by_occupation = top_100_mean_by_occupation.sort_values(by=["occupation","rating_mean"], ascending=False)
# 각 그룹별로 100개씩 가져오기
top_100_mean_by_occupation = top_100_mean_by_occupation.groupby("occupation").apply(top, column="rating_mean", asc=False, n=100)
top_100_mean_by_occupation

rating_mean
occupation    occupation    movie_title                                                    
administrator administrator A Chef in Love (1996)                                  5.000000
                            Shall We Dance? (1996)                                 5.000000
                            NeverEnding Story III, The (1994)                      5.000000
                            Once Were Warriors (1994)                              5.000000
                            Orlando (1993)                                         5.000000
                            Paradise Lost: The Child Murders at Robin Hood ...     5.000000
                            Paradise Road (1997)                                   5.000000
                            Pather Panchali (1955)                                 5.000000
                            Ponette (1996)                                         5.000000
                            Prefontaine (1997)                                     5.000000
                            Raise the Red Lantern (1991)                           5.000000
                            Ridicule (1996)                                        5.000000
                            Selena (1997)                                          5.000000
                            So Dear to My Heart (1949)                             5.000000
                            Mrs. Dalloway (1997)                                   5.000000
                            Stalingrad (1993)                                      5.000000
                            Tango Lesson, The (1997)                               5.000000
                            Trigger Effect, The (1996)                             5.000000
                            Truman Show, The (1998)                                5.000000
                            Umbrellas of Cherbourg, The (Parapluies de Cher...     5.000000
                            Unzipped (1995)                                        5.000000
                            Visitors, The (Visiteurs, Les) (1993)                  5.000000
                            Walking and Talking (1996)                             5.000000
                            When We Were Kings (1996)                              5.000000
                            Winter Guest, The (1997)                               5.000000
                            World of Apu, The (Apur Sansar) (1959)                 5.000000
                            My Family (1995)                                       5.000000
                            Aparajito (1956)                                       5.000000
                            Microcosmos: Le peuple de l'herbe (1996)               5.000000
                            Close Shave, A (1995)                                  5.000000
                            Assignment, The (1997)                                 5.000000
                            Audrey Rose (1977)                                     5.000000
                            Big Blue, The (Grand bleu, Le) (1988)                  5.000000
                            Blue Angel, The (Blaue Engel, Der) (1930)              5.000000
                            Bogus (1996)                                           5.000000
                            Inspector General, The (1949)                          5.000000
                            Boys of St. Vincent, The (1993)                        5.000000
                            Braindead (1992)                                       5.000000
                            Brassed Off (1996)                                     5.000000
                            Bread and Chocolate (Pane e cioccolata) (1973)         5.000000
                            Chairman of the Board (1998)                           5.000000
                            Boxing Helena (1993)                                   5.000000
                            Cong

In [333]:
movie_data = movie_data[["movie_title"]]
joined_genre_movie = pd.merge(left=movie_data, right=movie_genre, on="movie_id")
joined_genre_movie = pd.merge(left=joined_genre_movie, right=genre_info, on="genre_id").sort_values(by="movie_id").reset_index().drop(columns=["index"])
joined_genre_movie_user = pd.merge(left=user_data, right=rating_data, left_on="id", right_on="user_id")
joined_genre_movie_user = pd.merge(left=joined_genre_movie_user, right=joined_genre_movie, on="movie_id")
joined_genre_movie_user



,age,gender,occupation,zip_code,user_id,movie_id,rating,timestamp,movie_title,genre_id,genre_name
0,24,M,technician,85711,1,61,4,878542420,Three Colors: White (1994),8,Drama
1,47,M,educator,29206,13,61,4,882140552,Three Colors: White (1994),8,Drama
2,35,F,other,37212,18,61,4,880130803,Three Colors: White (1994),8,Drama
3,27,M,programmer,52246,58,61,5,884305271,Three Colors: White (1994),8,Drama
4,49,M,educator,08403,59,61,4,888204597,Three Colors: White (1994),8,Drama
...,...,...,...,...,...,...,...,...,...,...,...
212590,17,M,student,60089,863,1678,1,889289570,Mat' i syn (1997),8,Drama
212591,17,M,student,60089,863,1680,2,889289570,Sliding Doors (1998),14,Romance
212592,17,M,student,60089,863,1680,2,889289570,Sliding Doors (1998),8,Drama
212593,28,M,writer,91505,896,1681,3,887160722,You So Crazy (1994),5,Comedy


In [352]:
# 과제 3-A-1: 장르별 평가 수
rating_count_of_genre_all = joined_genre_movie_user.groupby("genre_name")[["rating"]].count()
rating_count_of_genre_all = rating_count_of_genre_all.rename(columns={"rating":"rating_count"})
rating_count_of_genre_all = rating_count_of_genre_all.sort_values(by="rating_count", ascending=False)
rating_count_of_genre_all

,rating_count
genre_name,
Drama,39895
Comedy,29832
Action,25589
Thriller,21872
Romance,19461
Adventure,13753
Sci-Fi,12730
War,9398
Crime,8055


In [351]:
# 과제 3-A-2: 성별, 장르별 평가 수
rating_count_of_genre_by_gender = joined_genre_movie_user.groupby(["gender","genre_name"])[["rating"]].count()
rating_count_of_genre_by_gender = rating_count_of_genre_by_gender.rename(columns={"rating":"rating_count"})
rating_count_of_genre_by_gender = rating_count_of_genre_by_gender.sort_values(by=["gender","rating_count"], ascending=False)
rating_count_of_genre_by_gender

rating_count
gender genre_name               
M      Drama               28887
       Comedy              21764
       Action              20147
       Thriller            16786
       Romance             13603
       Adventure           10612
       Sci-Fi              10101
       War                  7209
       Crime                6261
       Children's           4950
       Horror               4120
       Mystery              3931
       Musical              3512
       Animation            2610
       Western              1483
       Film-Noir            1348
       Fantasy               989
       Documentary           571
       unknown                 8
F      Drama               11008
       Comedy               8068
       Romance              5858
       Action               5442
       Thriller             5086
       Adventure            3141
       Sci-Fi               2629
       Children's           2232
       War                  2189
       Crime                1794
       Musical              1442
       Mystery              1314
       Horror               1197
       Animation             995
       Film-Noir             385
       Western               371
       Fantasy               363
       Documentary           187
       unknown                 2

In [356]:
# 과제 3-A-3: 직업별, 장르별 평가 수
rating_count_of_genre_by_occupation = joined_genre_movie_user.groupby(["occupation","genre_name"])[["rating"]].count()
rating_count_of_genre_by_occupation = rating_count_of_genre_by_occupation.rename(columns={"rating":"rating_count"})
rating_count_of_genre_by_occupation = rating_count_of_genre_by_occupation.sort_values(by=["occupation","rating_count"], ascending=False)
rating_count_of_genre_by_occupation

rating_count
occupation    genre_name               
writer        Drama                2208
              Comedy               1574
              Thriller             1304
              Action               1292
              Romance              1007
              Adventure             674
              Sci-Fi                671
              Crime                 498
              Horror                454
              War                   445
              Children's            367
              Mystery               335
              Musical               272
              Animation             176
              Film-Noir             125
              Western                94
              Fantasy                71
              Documentary            46
              unknown                 1
technician    Drama                1295
              Action               1079
              Comedy               1056
              Thriller              796
              Romance               645
              Adventure             558
              Sci-Fi                531
              War                   383
              Crime                 275
              Children's            221
              Mystery               177
              Musical               175
              Horror                164
              Animation             132
              Western                89
              Film-Noir              46
              Fantasy                38
              Documentary            24
              unknown                 1
student       Drama                7777
              Comedy               6958
              Action               6398
              Thriller             5130
              Romance              4156
              Adventure            3358
              Sci-Fi               3069
              Crime                1836
              War                  1770
              Children's           1759
              Horror               1340
              Musical              1078
              Mystery              1009
              Animation             914
              Western               408
              Fantasy               352
              Film-Noir             269
              Documentary           117
              unknown                 1
scientist     Drama                 932
              Action                566
              Comedy                535
              Thriller              469
              Romance               440
              Sci-Fi                318
              Adventure             285
              War                   255
              Crime                 177
              Mystery               122
              Children's            108
              Musical                91
              Horror                 70
              Animation              61
              Film-Noir              39
              Western                24
              Fantasy                20
              Documentary            17
salesman      Drama                 317
              Comedy                314
              Action                222
              Thriller              200
              Romance               166
              Adventure              99
              Sci-Fi                 98
              Crime                  79
              War                    77
              Horror                 49
              Mystery                40
              Children's             38
              Musical                27
              Western                16
              Animation              15
              Film-Noir              14
              Fantasy                 9
              Documentary             2
retired       Drama                 727
              Comedy                512
              Romance               321
              Thriller              309
              Action                278
              Adventure    

In [357]:
# 과제 3-B-1: 장르별 평점 평균
rating_mean_of_genre_all = joined_genre_movie_user.groupby("genre_name")[["rating"]].mean()
rating_mean_of_genre_all = rating_mean_of_genre_all.rename(columns={"rating":"rating_mean"})
rating_mean_of_genre_all = rating_mean_of_genre_all.sort_values(by="rating_mean", ascending=False)
rating_mean_of_genre_all

,rating_mean
genre_name,
Film-Noir,3.921523
War,3.815812
Drama,3.687379
Documentary,3.672823
Mystery,3.638132
Crime,3.632278
Romance,3.621705
Western,3.613269
Animation,3.576699


In [358]:
# 과제 3-B-2: 성별, 장르별 평점 평균
rating_mean_of_genre_by_gender = joined_genre_movie_user.groupby(["gender","genre_name"])[["rating"]].mean()
rating_mean_of_genre_by_gender = rating_mean_of_genre_by_gender.rename(columns={"rating":"rating_mean"})
rating_mean_of_genre_by_gender = rating_mean_of_genre_by_gender.sort_values(by=["gender","rating_mean"], ascending=False)
rating_mean_of_genre_by_gender

rating_mean
gender genre_name              
M      Film-Noir       3.973294
       War             3.826328
       Drama           3.696957
       Documentary     3.691769
       Mystery         3.664208
       Crime           3.654049
       Western         3.637896
       Romance         3.607072
       Sci-Fi          3.577072
       Animation       3.557471
       Thriller        3.512927
       Adventure       3.499246
       Action          3.479228
       Musical         3.472665
       Comedy          3.382972
       Children's      3.320000
       Horror          3.298058
       Fantasy         3.220425
       unknown         3.125000
F      War             3.781179
       Film-Noir       3.740260
       Drama           3.662246
       Romance         3.655685
       Musical         3.640083
       Animation       3.627136
       Documentary     3.614973
       Mystery         3.560122
       Crime           3.556299
       Adventure       3.517988
       Western         3.514825
       unknown         3.500000
       Sci-Fi          3.497908
       Thriller        3.496068
       Action          3.484013
       Children's      3.426971
       Comedy          3.424021
       Horror          3.263993
       Fantasy         3.201102

In [359]:
# 과제 3-B-3: 직업별, 장르별 평점 평균
rating_mean_of_genre_by_occupation = joined_genre_movie_user.groupby(["occupation","genre_name"])[["rating"]].mean()
rating_mean_of_genre_by_occupation = rating_mean_of_genre_by_occupation.rename(columns={"rating":"rating_mean"})
rating_mean_of_genre_by_occupation = rating_mean_of_genre_by_occupation.sort_values(by=["occupation","rating_mean"], ascending=False)
rating_mean_of_genre_by_occupation

rating_mean
occupation    genre_name              
writer        Film-Noir       3.968000
              Documentary     3.782609
              Mystery         3.629851
              War             3.577528
              Drama           3.511322
              Crime           3.502008
              Animation       3.500000
              Musical         3.474265
              Romance         3.469712
              Western         3.414894
              Thriller        3.375767
              Sci-Fi          3.347243
              Children's      3.324251
              Adventure       3.302671
              Action          3.285604
              Comedy          3.268742
              Fantasy         3.197183
              Horror          3.107930
              unknown         1.000000
technician    Film-Noir       4.021739
              unknown         4.000000
              Documentary     3.958333
              War             3.715405
              Drama           3.694981
              Mystery         3.689266
              Romance         3.631008
              Sci-Fi          3.630885
              Animation       3.606061
              Crime           3.509091
              Thriller        3.488693
              Western         3.483146
              Action          3.459685
              Adventure       3.458781
              Musical         3.457143
              Horror          3.420732
              Comedy          3.381629
              Children's      3.271493
              Fantasy         3.157895
student       Film-Noir       3.832714
              Documentary     3.829060
              War             3.819209
              Drama           3.702842
              Crime           3.657407
              Romance         3.650866
              Animation       3.563457
              Mystery         3.555005
              Western         3.549020
              Sci-Fi          3.531118
              Thriller        3.506238
              Musical         3.501855
              Adventure       3.484515
              Action          3.466552
              Horror          3.389552
              Comedy          3.375683
              Children's      3.356453
              Fantasy         3.048295
              unknown         3.000000
scientist     Film-Noir       4.025641
              Documentary     3.882353
              War             3.815686
              Drama           3.815451
              Crime           3.813559
              Mystery         3.729508
              Animation       3.639344
              Western         3.625000
              Romance         3.615909
              Sci-Fi          3.578616
              Musical         3.549451
              Thriller        3.496802
              Children's      3.481481
              Comedy          3.446729
              Adventure       3.442105
              Action          3.422261
              Horror          3.385714
              Fantasy         3.150000
salesman      Documentary     4.000000
              Drama           3.984227
              War             3.948052
              Western         3.937500
              Crime           3.860759
              Film-Noir       3.785714
              Mystery         3.775000
              Adventure       3.717172
              Animation       3.666667
              Action          3.653153
              Romance         3.632530
              Musical         3.592593
              Thriller        3.585000
              Sci-Fi          3.540816
              Children's      3.500000
              Fantasy         3.333333
              Horror          3.326531
              Comedy          3.187898
retired       Film-Noir       4.023256
              Mystery         3.714286
              Animation       3.628571
              War             3.627586
              Crime           3.613139
              Musical         3.526882
              Western         3.515152
              Drama           3.514443
          